# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port douglas,-16.4833,145.4667,23.36,66,2,3.09,AU,2023-07-22
1,1,port-aux-francais,-49.3500,70.2167,0.70,96,100,1.52,TF,2023-07-22
2,2,hanjiashu,39.1875,117.0731,23.98,51,9,3.22,CN,2023-07-22
3,3,waitangi,-43.9535,-176.5597,12.12,91,100,3.13,NZ,2023-07-22
4,4,puerto ayora,-0.7393,-90.3518,24.57,81,94,4.43,EC,2023-07-22


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# im using hvpot to show the interactive map 
# using Lng and Lat to plot cities on the map
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600,
    color="City",
    size="Humidity",
    alpha=0.5)

# Display the map

cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
# 
fit_city_df = city_data_df.loc[(city_data_df["Max Temp"]< 25)
                                &(city_data_df["Max Temp"]>20)
                                &(city_data_df["Wind Speed"] < 4.5)
                                &(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
#
fit_city_df.dropna()

# Display sample data
# YOUR CODE HERE
fit_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,portland,45.5234,-122.6762,22.99,60,0,2.57,US,2023-07-22
85,85,bethel,41.3712,-73.4140,20.36,87,0,0.00,US,2023-07-22
177,177,petropavlovsk-kamchatsky,53.0452,158.6483,22.33,64,0,3.00,RU,2023-07-22
193,193,fremantle,-32.0500,115.7667,20.88,51,0,1.54,AU,2023-07-22
227,227,spring creek,40.7266,-115.5859,22.39,17,0,1.81,US,2023-07-22


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity",]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"]=""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,san patricio,US,28.0170,-97.5169,84,
33,portland,US,45.5234,-122.6762,60,
61,tindouf,DZ,27.6711,-8.1474,26,
85,bethel,US,41.3712,-73.4140,87,
177,petropavlovsk-kamchatsky,RU,53.0452,158.6483,64,
193,fremantle,AU,-32.0500,115.7667,51,
221,cruz bay,VI,18.3313,-64.7937,83,
227,spring creek,US,40.7266,-115.5859,17,
237,aracati,BR,-4.5617,-37.7697,78,
238,santa ana,US,33.7456,-117.8678,90,


In [35]:
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


san patricio - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
tindouf - nearest hotel: محمد بوسبي
bethel - nearest hotel: Hampton Inn Danbury
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
fremantle - nearest hotel: Tradewinds Hotel
cruz bay - nearest hotel: Inn at Tamarind Court
spring creek - nearest hotel: No hotel found
aracati - nearest hotel: Hotel Complexo Mirante das Gamboas
santa ana - nearest hotel: Holiday Inn Express
luis correia - nearest hotel: Hotel Amaro
palmas - nearest hotel: Hotel Triângulo Mineiro
geraldton - nearest hotel: Ocean Centre Hotel
badger - nearest hotel: Hotel North Pole
uvalde - nearest hotel: No hotel found
sudak - nearest hotel: Владислав
mpongwe - nearest hotel: No hotel found
xining - nearest hotel: Qinghai hotel
barcelos - nearest hotel: Hotel Dom Nuno
eregli - nearest hotel: No hotel found
kagizman - nearest hotel: No hotel found
arkalyk - nearest hotel: Гостиница "Арсенал"
sinop - nearest hotel: No hotel found
we

,City,Country,Lat,Lng,Humidity,Hotel Name
23,san patricio,US,28.0170,-97.5169,84,No hotel found
33,portland,US,45.5234,-122.6762,60,the Hoxton
61,tindouf,DZ,27.6711,-8.1474,26,محمد بوسبي
85,bethel,US,41.3712,-73.4140,87,Hampton Inn Danbury
177,petropavlovsk-kamchatsky,RU,53.0452,158.6483,64,ООО Постоялый двор
193,fremantle,AU,-32.0500,115.7667,51,Tradewinds Hotel
221,cruz bay,VI,18.3313,-64.7937,83,Inn at Tamarind Court
227,spring creek,US,40.7266,-115.5859,17,No hotel found
237,aracati,BR,-4.5617,-37.7697,78,Hotel Complexo Mirante das Gamboas
238,santa ana,US,33.7456,-117.8678,90,Holiday Inn Express


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san patricio - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
tindouf - nearest hotel: محمد بوسبي
bethel - nearest hotel: Hampton Inn Danbury
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
fremantle - nearest hotel: Tradewinds Hotel
cruz bay - nearest hotel: Inn at Tamarind Court
spring creek - nearest hotel: No hotel found
aracati - nearest hotel: Hotel Complexo Mirante das Gamboas
santa ana - nearest hotel: Holiday Inn Express
luis correia - nearest hotel: Hotel Amaro
palmas - nearest hotel: Hotel Triângulo Mineiro
geraldton - nearest hotel: Ocean Centre Hotel
badger - nearest hotel: Hotel North Pole
uvalde - nearest hotel: No hotel found
sudak - nearest hotel: Владислав
mpongwe - nearest hotel: No hotel found
xining - nearest hotel: Qinghai hotel
barcelos - nearest hotel: Hotel Dom Nuno
eregli - nearest hotel: No hotel found
kagizman - nearest hotel: No hotel found
arkalyk - nearest hotel: Гостиница "Арсенал"
sinop - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
23,san patricio,US,28.0170,-97.5169,84,No hotel found
33,portland,US,45.5234,-122.6762,60,the Hoxton
61,tindouf,DZ,27.6711,-8.1474,26,محمد بوسبي
85,bethel,US,41.3712,-73.4140,87,Hampton Inn Danbury
177,petropavlovsk-kamchatsky,RU,53.0452,158.6483,64,ООО Постоялый двор
193,fremantle,AU,-32.0500,115.7667,51,Tradewinds Hotel
221,cruz bay,VI,18.3313,-64.7937,83,Inn at Tamarind Court
227,spring creek,US,40.7266,-115.5859,17,No hotel found
237,aracati,BR,-4.5617,-37.7697,78,Hotel Complexo Mirante das Gamboas
238,santa ana,US,33.7456,-117.8678,90,Holiday Inn Express


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles= "OSM",
    frame_width = 800,
    frame_height= 600,
    color="City",
    size = "Humidity",
    alpha = .70,
    hover_cols = ["Hotel Name","Country"])
# Display the map
# YOUR CODE HERE
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)